In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import pickle

In [9]:
Archivos = "../Data"
df_homicidios = pd.read_excel(Archivos+"/HOMICIDIO.xlsx")

# nueva columna que represente si la cantidad de homicidios va a subir (1) o bajar (0)
df_homicidios['CANTIDAD_CAMBIO'] = np.where(df_homicidios['CANTIDAD'].diff() > 0, 1, 0)

# Dividir los datos en características (X) y la variable objetivo (y)
X = df_homicidios[['COD_DEPTO', 'COD_MUNI', 'SEXO', 'ZONA','CANTIDAD']]
y = df_homicidios['CANTIDAD_CAMBIO']

# Convertir variables categóricas a variables dummy
X = pd.get_dummies(X, columns=['SEXO', 'ZONA'], drop_first=True)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#X.head()

# Utiliza los mejores hiperparámetros encontrados
best_params = {'n_estimators': 150, 'min_samples_split': 15, 'min_samples_leaf': 4, 'max_depth': 10}

# Crear el modelo RandomForestClassifier con los mejores hiperparámetros encontrados
best_clf = RandomForestClassifier(random_state=42, **best_params,n_jobs=-1)

# Entrenar el modelo con todos los datos disponibles
best_clf.fit(X, y)

# Realizar predicciones en un conjunto de datos si es necesario
y_pred = best_clf.predict(X_test)  # Puedes usar X_test si quieres predecir en datos de prueba

# Evaluar el rendimiento del modelo si corresponde
accuracy = accuracy_score(y_test, y_pred)  # Si tienes datos de prueba y su correspondiente 'y_test'
print(f'Accuracy: {accuracy}')

# Ver un informe de clasificación si lo necesitas
print(classification_report(y_test, y_pred))  # Si tienes datos de prueba y su correspondiente 'y_test'


C:\Users\JD_01\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\JD_01\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\JD_01\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\JD_01\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed

Accuracy: 0.9807846992582
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     37593
           1       0.90      1.00      0.94      7163

    accuracy                           0.98     44756
   macro avg       0.95      0.99      0.97     44756
weighted avg       0.98      0.98      0.98     44756



In [10]:
# Guardar el modelo en un archivo pickle
with open("rf_model.pkl", "wb") as model_file:
    pickle.dump(best_clf, model_file)

In [46]:
y

0         0
1         0
2         0
3         1
4         0
         ..
223773    0
223774    0
223775    0
223776    0
223777    0
Name: CANTIDAD_CAMBIO, Length: 223778, dtype: int32

In [35]:
best_clf.feature_importances_

array([2.29357558e-02, 4.97946604e-02, 9.08288751e-01, 7.15140799e-03,
       3.64248587e-06, 1.18257821e-02])

In [40]:
datos = {
    "COD_DEPTO": [91],
    "DEPARTAMENTO": ["AMAZONAS"],
    "COD_MUNI": [91001],
    "MUNICIPIO": ["LETICIA"],
    "SEXO": ["MASCULINO"],
    "ZONA": ["RURAL"],
    "CANTIDAD": [1],
}
df = pd.DataFrame(datos)
test = df[['COD_DEPTO', 'COD_MUNI', 'SEXO', 'ZONA','CANTIDAD']]
test = pd.get_dummies(test, columns=['SEXO', 'ZONA'])
print(test)
features_to_check = ["SEXO_NO REPORTA", "ZONA_URBANA", "SEXO_MASCULINO"]

# Verificar si las características están presentes y, si no, agregarlas con valores falsos
for feature in features_to_check:
    if feature not in test.columns:
        test[feature] = False
#test.head()
Lista = ["COD_DEPTO", "COD_MUNI", "CANTIDAD", "SEXO_MASCULINO", "SEXO_NO REPORTA", "ZONA_URBANA"]
test = test[Lista]
best_clf.predict(test)

   COD_DEPTO  COD_MUNI  CANTIDAD  SEXO_MASCULINO  ZONA_RURAL
0         91     91001         1            True        True


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- CANTIDAD


In [15]:
df_homicidios['ZONA'].value_counts()

ZONA
URBANA    142938
RURAL      80840
Name: count, dtype: int64

In [27]:
df_homicidios['SEXO'].value_counts()

SEXO
MASCULINO     200729
FEMENINO       22930
NO REPORTA       119
Name: count, dtype: int64

In [37]:
feature_importances = best_clf.feature_importances_

# Obtener nombres de las características
feature_names = X_train.columns  # Asumiendo que estás usando un DataFrame de pandas

# Imprimir las importancias de las características
for feature, importance in zip(feature_names, feature_importances):
    print(f"{feature}: {importance}")

# O simplemente imprimir el ranking de importancia
importances_ranked = np.argsort(feature_importances)[::-1]
for idx in importances_ranked:
    print(f"{feature_names[idx]}: {feature_importances[idx]}")

COD_DEPTO: 0.02293575579715134
COD_MUNI: 0.0497946603583057
CANTIDAD: 0.9082887513069446
SEXO_MASCULINO: 0.007151407992675713
SEXO_NO REPORTA: 3.6424858704016224e-06
ZONA_URBANA: 0.01182578205905236
CANTIDAD: 0.9082887513069446
COD_MUNI: 0.0497946603583057
COD_DEPTO: 0.02293575579715134
ZONA_URBANA: 0.01182578205905236
SEXO_MASCULINO: 0.007151407992675713
SEXO_NO REPORTA: 3.6424858704016224e-06
